<a href="https://colab.research.google.com/github/sornavel/Hackathons-Janatahack/blob/master/Healthcare_CAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyDrive

In [ ]:
import os
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
np.random.seed(0)
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()
from IPython.core.display import HTML 
from IPython.display import Image
import gc
from keras.applications.densenet import preprocess_input, DenseNet121
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
import keras.backend as K

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download = drive.CreateFile({'id': '1FUu6ZyHdGB_ElcE4ZuwzPAh-DbQ7mAwQ'})
download.GetContentFile('train.csv')
train_df = pd.read_csv('train.csv')

download = drive.CreateFile({'id': '1vE_M60f3HJ4R1GSFl3QjHWzuwEIcVrmd'})
download.GetContentFile('Third_Health_Camp_Attended.csv')
third_df = pd.read_csv('Third_Health_Camp_Attended.csv')

download = drive.CreateFile({'id': '1hnoPLF_uC4hrpCcSFmGJ1UxLog7grgrl'})
download.GetContentFile('Second_Health_Camp_Attended.csv')
second_df = pd.read_csv('Second_Health_Camp_Attended.csv')

download = drive.CreateFile({'id': '1LMzFdyra4gObQctf5h0lb6DtUnqiLBT3'})
download.GetContentFile('First_Health_Camp_Attended.csv')
first_df = pd.read_csv('First_Health_Camp_Attended.csv')

download = drive.CreateFile({'id': '16JMb9xNz5BEMlV9dS7reh9yi8ivGHYT2'})
download.GetContentFile('Patient_Profile.csv')
patient_df = pd.read_csv('Patient_Profile.csv')

download = drive.CreateFile({'id': '16AFNM3SlDDRMpEAdq4ssddK5QwoH9S33'})
download.GetContentFile('Health_Camp_Detail.csv')
camp_df = pd.read_csv('Health_Camp_Detail.csv')

download = drive.CreateFile({'id': '1aQqKgHFJ0M5j-V34G_TZgOIi2xc6GnaA'})
download.GetContentFile('test.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0
3,524931,6535,07-Feb-04,0,0,0,0,0
4,521364,6529,28-Feb-06,15,1,0,0,7


In [ ]:
first_df.head()

,Patient_ID,Health_Camp_ID,Donation,Health_Score,Unnamed: 4
0,506181,6560,40,0.439024,NaN
1,494977,6560,20,0.097561,NaN
2,518680,6560,10,0.048780,NaN
3,509916,6560,30,0.634146,NaN
4,488006,6560,20,0.024390,NaN


In [ ]:
second_df.head()

,Patient_ID,Health_Camp_ID,Health Score
0,526631,6536,0.875136
1,509122,6536,0.755700
2,498864,6536,0.673181
3,515398,6536,0.722041
4,504624,6536,0.464712


In [ ]:
third_df.head()

,Patient_ID,Health_Camp_ID,Number_of_stall_visited,Last_Stall_Visited_Number
0,517875,6527,3,1
1,504692,6578,1,1
2,504692,6527,3,1
3,493167,6527,4,4
4,510954,6528,2,2


In [ ]:
patient_df.head()

,Patient_ID,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,516956,0,0,0,0,1,90,39,18-Jun-03,NaN,Software Industry
1,507733,0,0,0,0,1,None,40,20-Jul-03,H,Software Industry
2,508307,0,0,0,0,3,87,46,02-Nov-02,D,BFSI
3,512612,0,0,0,0,1,75,47,02-Nov-02,D,Education
4,521075,0,0,0,0,3,None,80,24-Nov-02,H,Others


In [ ]:
camp_df.head()

,Health_Camp_ID,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,6560,16-Aug-03,20-Aug-03,First,B,2
1,6530,16-Aug-03,28-Oct-03,First,C,2
2,6544,03-Nov-03,15-Nov-03,First,F,1
3,6585,22-Nov-03,05-Dec-03,First,E,2
4,6561,30-Nov-03,18-Dec-03,First,E,1


In [ ]:
test_df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,505701,6548,21-May-06,1,0,0,0,2
1,500633,6584,02-Jun-06,0,0,0,0,0
2,506945,6582,10-Aug-06,0,0,0,0,0
3,497447,6551,27-Aug-06,0,0,0,0,0
4,496446,6533,19-Sep-06,0,0,0,0,0


In [ ]:
third_df_filtered = third_df[third_df.Number_of_stall_visited!=0]

In [ ]:
df = pd.concat([train_df,test_df],axis=0)
df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0
3,524931,6535,07-Feb-04,0,0,0,0,0
4,521364,6529,28-Feb-06,15,1,0,0,7


In [ ]:
temp = pd.merge(df,first_df,how='left', on=['Patient_ID','Health_Camp_ID'])
temp.Health_Score.loc[~temp.Health_Score.isnull()] = 1
temp.Health_Score.loc[temp.Health_Score.isnull()] = 0
df = temp.rename(columns={'Health_Score':'succ_1'})

temp = pd.merge(df,second_df,how='left', on=['Patient_ID','Health_Camp_ID'])
temp['Health Score'].loc[~temp['Health Score'].isnull()] = 1
temp['Health Score'].loc[temp['Health Score'].isnull()] = 0
df = temp.rename(columns={'Health Score':'succ_2'})

temp = pd.merge(df,third_df_filtered,how='left', on=['Patient_ID','Health_Camp_ID'])
temp['Number_of_stall_visited'].loc[~temp['Number_of_stall_visited'].isnull()] = 1
temp['Number_of_stall_visited'].loc[temp['Number_of_stall_visited'].isnull()] = 0
df = temp.rename(columns={'Number_of_stall_visited':'succ_3'})
df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Donation,succ_1,Unnamed: 4,succ_2,succ_3,Last_Stall_Visited_Number
0,489652,6578,10-Sep-05,4,0,0,0,2,NaN,0.0,NaN,0.0,1.0,1.0
1,507246,6578,18-Aug-05,45,5,0,0,7,NaN,0.0,NaN,0.0,0.0,NaN
2,523729,6534,29-Apr-06,0,0,0,0,0,NaN,0.0,NaN,1.0,0.0,NaN
3,524931,6535,07-Feb-04,0,0,0,0,0,NaN,0.0,NaN,0.0,0.0,NaN
4,521364,6529,28-Feb-06,15,1,0,0,7,NaN,0.0,NaN,1.0,0.0,NaN


In [ ]:
df.drop(['Donation','Unnamed: 4','Last_Stall_Visited_Number'],axis=1,inplace=True)

In [ ]:
df['Reg_year'] = pd.DatetimeIndex(df.Registration_Date).year
df['Reg_month'] = pd.DatetimeIndex(df.Registration_Date).month
df['Reg_day'] = pd.DatetimeIndex(df.Registration_Date).day

In [ ]:
temp = pd.merge(df,patient_df,on='Patient_ID',how='left')
df=temp
temp = pd.merge(df,camp_df,on='Health_Camp_ID',how='left')
df=temp

In [ ]:
df.drop(['Patient_ID','Health_Camp_ID','Registration_Date',],axis=1,inplace=True)

In [ ]:
df.Income[df.Income=='None'] = 0
df.Education_Score[df.Education_Score=='None'] = 0
df.Age[df.Age=='None'] = 0
df.Age = df.Age.astype('int64')
df.Age[df.Age==0] = df.Age[df.Age!=0].mean()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [ ]:
df['fi_year'] = pd.DatetimeIndex(df.First_Interaction).year
df['fi_month'] = pd.DatetimeIndex(df.First_Interaction).month
df['fi_day'] = pd.DatetimeIndex(df.First_Interaction).day

In [ ]:
df.drop(['First_Interaction'],axis=1,inplace=True)

In [ ]:
#temp = df.City_Type[~df.City_Type.isnull()].mode()
df.City_Type = df.City_Type.replace(np.nan,'B')
df.Employer_Category = df.Employer_Category.replace(np.nan,'Technology')

In [ ]:
temp = pd.to_datetime(df.Camp_End_Date) - pd.to_datetime(df.Camp_Start_Date)
df['camp_days'] = temp.dt.days

In [ ]:
df.drop(['Camp_Start_Date', 'Camp_End_Date'],axis=1,inplace=True)
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 
df['City_Type']= le.fit_transform(df['City_Type']) 
df['Category1']= le.fit_transform(df['Category1']) 
df['Category2']= le.fit_transform(df['Category2']) 
df['Employer_Category']= le.fit_transform(df['Employer_Category']) 


In [ ]:
#temp = df.Reg_year[~df.Reg_year.isnull()].mode() = 2006
df.Reg_year = df.Reg_year.replace(np.nan,2006)
df.Reg_month = df.Reg_month.replace(np.nan,9)
df.Reg_day = df.Reg_day.replace(np.nan,18)

In [ ]:
df['succ'] = df['succ_1'] + df['succ_2'] + df['succ_3']

In [ ]:
y_train = df['succ'][:train_df.shape[0]]
df.drop(['succ','succ_1','succ_2','succ_3'],axis=1,inplace = True)

In [ ]:
df.head()

,Var1,Var2,Var3,Var4,Var5,Reg_year,Reg_month,Reg_day,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2,Category3,fi_year,fi_month,fi_day,camp_days
0,4,0,0,0,2,2005.0,9.0,10.0,0,0,0,0,0,0,48.20876,1,11,2,6,2,2004,12,6,59
1,45,5,0,0,7,2005.0,8.0,18.0,0,0,0,0,1,75,40.00000,2,7,2,6,2,2004,9,8,59
2,0,0,0,0,0,2006.0,4.0,29.0,0,0,0,0,0,0,48.20876,1,11,1,0,2,2004,6,22,751
3,0,0,0,0,0,2004.0,2.0,7.0,0,0,0,0,0,0,48.20876,8,11,0,4,2,2004,2,7,17
4,15,1,0,0,7,2006.0,2.0,28.0,0,0,0,1,1,70,40.00000,8,11,1,0,2,2003,7,4,4


In [ ]:
df.Income = df.Income.astype('int64')
df.Education_Score = df.Education_Score.astype('float64')

In [ ]:
for col in ['Online_Follower', 'LinkedIn_Shared', 'Twitter_Shared', 'Facebook_Shared', 'City_Type','Employer_Category','Category1','Category2','Category3']:
    df[col] = df[col].astype('category')

In [ ]:
train = df[:train_df.shape[0]]
label = y_train
test = df[train_df.shape[0]:]
X = df[:train_df.shape[0]]
y = label
Xtest = test

In [ ]:
print(X.shape,Xtest.shape)
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state = 1994,stratify=y)
categorical_features_indices = np.where(X_train.dtypes =='category')[0]
categorical_features_indices

(75278, 24) (35249, 24)


array([ 8,  9, 10, 11, 15, 16, 17, 18, 19])

In [ ]:
!pip3 install catboost
from catboost import CatBoostClassifier
m=CatBoostClassifier(n_estimators=2500,random_state=1994,learning_rate=0.03,eval_metric='AUC')
# m=RidgeCV(cv=4)
m.fit(X_train,y_train,eval_set=[(X_val, y_val.values)], early_stopping_rounds=300,verbose=200,cat_features=categorical_features_indices)
p=m.predict_proba(X_val)[:,-1]

0:	test: 0.7782188	best: 0.7782188 (0)	total: 130ms	remaining: 5m 25s
200:	test: 0.8445114	best: 0.8445114 (200)	total: 13.8s	remaining: 2m 38s
400:	test: 0.8496269	best: 0.8496269 (400)	total: 27.4s	remaining: 2m 23s
600:	test: 0.8531331	best: 0.8531331 (600)	total: 40.9s	remaining: 2m 9s
800:	test: 0.8550958	best: 0.8551012 (797)	total: 54.4s	remaining: 1m 55s
1000:	test: 0.8564777	best: 0.8564777 (1000)	total: 1m 8s	remaining: 1m 42s
1200:	test: 0.8575066	best: 0.8575066 (1200)	total: 1m 22s	remaining: 1m 29s
1400:	test: 0.8582272	best: 0.8582272 (1400)	total: 1m 36s	remaining: 1m 15s
1600:	test: 0.8588679	best: 0.8588679 (1600)	total: 1m 50s	remaining: 1m 1s
1800:	test: 0.8594137	best: 0.8594137 (1800)	total: 2m 4s	remaining: 48.2s
2000:	test: 0.8599801	best: 0.8599870 (1995)	total: 2m 18s	remaining: 34.5s
2200:	test: 0.8604900	best: 0.8604934 (2198)	total: 2m 32s	remaining: 20.7s
2400:	test: 0.8609578	best: 0.8609633 (2398)	total: 2m 46s	remaining: 6.87s
2499:	test: 0.8610934	best

NameError: ignored

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
print(roc_auc_score(y_val,p))

0.8610935961959654


In [ ]:
errCB=[]
y_pred_tot_cb=[]
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=15,shuffle=True,random_state=1994)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=CatBoostClassifier(n_estimators=5000,random_state=1994,eval_metric='AUC',learning_rate=0.03)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=200,cat_features=categorical_features_indices)
    preds=m.predict_proba(X_test)[:,-1]
    print("err_cb: ",roc_auc_score(y_test,preds))
    errCB.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(Xtest)[:,-1]
    i=i+1
    y_pred_tot_cb.append(p)

0:	test: 0.7900731	test1: 0.7950452	best: 0.7950452 (0)	total: 119ms	remaining: 9m 56s
200:	test: 0.8484278	test1: 0.8504699	best: 0.8504931 (199)	total: 19.8s	remaining: 7m 53s
400:	test: 0.8574015	test1: 0.8563980	best: 0.8563980 (400)	total: 39.9s	remaining: 7m 37s
600:	test: 0.8636233	test1: 0.8592748	best: 0.8592748 (600)	total: 1m	remaining: 7m 22s
800:	test: 0.8675647	test1: 0.8605430	best: 0.8605605 (794)	total: 1m 20s	remaining: 7m 2s
1000:	test: 0.8708679	test1: 0.8619523	best: 0.8619729 (997)	total: 1m 40s	remaining: 6m 42s
1200:	test: 0.8737525	test1: 0.8628725	best: 0.8628753 (1199)	total: 2m 1s	remaining: 6m 22s
1400:	test: 0.8763818	test1: 0.8636663	best: 0.8636703 (1399)	total: 2m 21s	remaining: 6m 4s
1600:	test: 0.8785573	test1: 0.8642135	best: 0.8642173 (1599)	total: 2m 42s	remaining: 5m 45s
1800:	test: 0.8806384	test1: 0.8646946	best: 0.8646946 (1800)	total: 3m 3s	remaining: 5m 26s
2000:	test: 0.8823390	test1: 0.8652051	best: 0.8652429 (1988)	total: 3m 24s	remaining:

In [ ]:
np.mean(errCB,0)

0.866223144923629

In [ ]:
sub_df = pd.DataFrame(test_df)
sub_df["Outcome"] = np.mean(y_pred_tot_cb,0)
sub_final_df = test_df[['Patient_ID','Health_Camp_ID','Outcome']]
sub_final_df.head()

,Patient_ID,Health_Camp_ID,Outcome
0,505701,6548,0.680306
1,500633,6584,0.559970
2,506945,6582,0.078978
3,497447,6551,0.859883
4,496446,6533,0.054042


In [ ]:
sub_final_df.to_csv("submission.csv", index=False)

In [85]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>